In [1]:
"""
Here we use the fasttext embeddings and 
convert our sentences into 300 dimensional vectors
"""
import numpy as np
import pandas as pd
from numpy import save
from numpy import load

from sklearn import model_selection

import nltk
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')

import re, string
import io
import sys

import time

In [2]:
# Read trainig data
df= pd.read_hdf('../input/tiny_data/train_tiny.h5',key='dataset')
df = df[["PostId","Title","BodyMarkdown","OpenStatus"]]
y = df.OpenStatus.values

In [3]:
df

,PostId,Title,BodyMarkdown,OpenStatus
2082369,7930515,How to get Entities based on field values in a...,I have entity `Product` and entity `Subcategor...,open
2223774,8355593,"Can you create several ""springSecurityFilterCh...",I'm configuring a web application to use sprin...,open
2869377,10024009,Edit and Add New in joomla custom component co...,I've started to make my own custom component f...,open
327831,1820877,"SQL Server, find an arbitrary sequence of values",Let's assume we have a table Maintenance\r\n\r...,open
596214,2893377,Question about pipe in Perl,The following code is working sort of fine:\r\...,open
...,...,...,...,...
1359908,5621144,Binary compatibility of FILE*...,I am designing C library which does some mathe...,open
1495972,6071804,Is it possible to add a special row in android...,I added the Search Dialog (Quick Search Box) t...,open
1100348,4718427,Ghostscript - PS to PDF - Inverted images problem,I'm trying to convert postscript to PDF using ...,open
2486007,9125136,Drupal service module Json Parser,I am using Drupal6 and try to implement Servic...,open


In [13]:
#punctuation = list(string.punctuation)

my_stopwords = nltk.corpus.stopwords.words('english')# punctuation

re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet =str(tweet).lower()
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def tokenize(s):
    s=str(s).lower()
    output = re.sub(r'\d+', '', s) # remove numbers 
    output = remove_links(output)
    result = re_tok.sub(r' \1 ', output).split() 
    result = [word for word in result if len(word)>2]
    result = [word for word in result if word not in my_stopwords]
    return result

# convert sentences to vectors from embedding, embedding is 300 dimensional
def sentence_to_vec(s,embedding_dict,stop_words,tokenizer):
    """
    s: sentence, string
    embedding_dict: dictionary word: vector
    stop_words: list of stop words
    tokenizer: tokenizer function
    """
    # tokenize the sentence
    words = s
    words = tokenizer(words)
    
    # keep only alpha numeric tokens
    words =[w for w in words if w.isalpha()]
    # initialize empty list to store embeddings
    M = []
    for w in words:
        # for every word, get the embedding from the dictionary
        # and append to the list of embeddings
        if w in embedding_dict:
            M.append(embedding_dict[w])
    # if we don't have any vectors return zeros
    if len(M)==0:
        return np.zeros(300)
    # convert list of embeddings to array
    M = np.array(M)
    # calculate sum over axis=0
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())       


def load_vectors(fname):
    fin = io.open(fname,'r',
                  encoding ='utf-8',
                  newline = '\n',
                  errors='ignore')
    n,d = map(int,fin.readline().split())
    data ={}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]]=list(map(float,tokens[1:]))
    return data  

def loadGloveModel(File):
    print("Loading Glove Model")
    
    f = io.open(File,'r',
                encoding='utf-8',
                newline = '\n',
                errors = 'ignore')
    
    gloveModel = {}
    for line in f:
        splitLines = line.split(' ')
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel


#create sentence embeddings
def get_vectors(vectors,column,embeddings):
    """
    vectors: empty array to fill in
    column: dataframe column
    return embedding vectors
    """
    for body in column:
        vectors.append(
            sentence_to_vec(s= body,
                           embedding_dict = embeddings,
                           stop_words =my_stopwords,
                           tokenizer=tokenize
                           )
        )
    return vectors  

In [14]:
 t0_total =time.time()

    # Read embeddings
t0=time.time()
print("Loading embeddings")
embeddings = loadGloveModel("../input/embeddings/glove.840B.300d.txt")  
    
t1 =time.time()
total_time=t1-t0
print("time to load", total_time)
print(" ")

Loading embeddings
Loading Glove Model
2196016  words loaded!
time to load 101.03192806243896
 


In [15]:
def get_sentence_embedding(df,column=None):    
    """
    param
    df: dataframe
    column: datafrane column to embedd
    return: dataframe with embedding.dim (#of rows, 301)
    """
    #Create empty dataframe
    
    data=pd.DataFrame([])      
        
    # loop over the postid
    
    for i in df.PostId:
        B=df.loc[df.PostId==i]    
        B=B[column].values
        # get the embeddings
        X= sentence_to_vec(s=[B],
                           embedding_dict = embeddings,
                           stop_words =my_stopwords,
                           tokenizer=tokenize
                           )
        
        data1=pd.DataFrame(data=X)
        data1=data1.T
        data1=data1.add_suffix('_'+column)
        #add id column
        data1["id"]=i
        data1.columns=data1.columns.astype(str)
        data=data.append(data1)        
    return data

def get_target(df,column=None):
    
    data=pd.DataFrame([])
    
    for i in df.PostId:
        B=df.loc[df.PostId==i]    
        B=B[column].values
        data1=pd.DataFrame(data=B)
        data1=data1.T
        #add id column
        data1["id"]=i
        data1.columns=data1.columns.astype(str)
        data1=data1.rename(columns={'0':'target'})
        data=data.append(data1)        
    return data
        

In [17]:
title_appended = get_sentence_embedding(df,'Title') 
print(title_appended.shape)
title_appended.to_hdf("../input/tiny_data/title_glove.h5",
         key='dataset',mode ='w',index= False)
(title_appended.head(7))

(26965, 301)


,0_Title,1_Title,2_Title,3_Title,4_Title,5_Title,6_Title,7_Title,8_Title,9_Title,...,291_Title,292_Title,293_Title,294_Title,295_Title,296_Title,297_Title,298_Title,299_Title,id
0,0.073908,-0.060587,0.011733,0.014747,0.024476,0.049523,-0.014929,0.033625,-0.015833,0.401764,...,0.036531,-0.053305,0.010568,-0.060879,-0.045012,-0.048568,-0.004205,-0.004861,-0.000387,7930515
0,0.085331,-0.106435,-0.020397,-0.007024,0.020592,0.068995,0.012632,0.035522,-0.038624,0.229388,...,0.054559,-0.022033,0.021855,-0.071808,-0.035656,-0.019580,0.001450,-0.002711,0.041868,8355593
0,0.073898,-0.057373,0.000684,0.012980,0.022916,0.059403,0.006969,-0.010689,-0.056265,0.220766,...,0.039930,-0.059229,0.032981,-0.047012,-0.061615,-0.062032,-0.036874,-0.025163,0.065984,10024009
0,0.052392,-0.054337,-0.012554,0.024567,0.050308,0.104697,0.018220,0.060285,0.038109,0.186173,...,0.033116,-0.057895,0.056810,-0.002417,-0.044769,-0.073489,-0.012145,-0.030923,0.000228,1820877
0,0.095446,-0.075444,-0.015266,0.027685,0.020788,0.071355,0.071580,0.028390,-0.021257,0.096025,...,-0.008491,-0.064513,0.030983,0.011050,0.004907,-0.042573,0.017142,-0.034573,0.064025,2893377
0,0.055459,-0.020370,-0.015782,0.025507,0.026345,0.044763,0.025692,-0.001153,-0.035493,0.145603,...,0.068572,-0.082965,0.022342,-0.014543,-0.081629,-0.080799,-0.019973,-0.003107,0.023808,6625356
0,0.060387,-0.029623,0.007930,0.008026,0.015444,0.090687,0.052027,0.015147,0.005541,0.135588,...,0.037266,-0.082607,0.028847,-0.026970,-0.047775,-0.085618,-0.033330,-0.028030,0.033308,9161866


In [142]:
BodyMarkDown_appended = get_sentence_embedding(df,'BodyMarkdown')
print(BodyMarkDown_appended.shape)
BodyMarkDown_appended.to_hdf(
    "../input/tiny_data/BodyMarkDown_glove.h5",
    key='dataset',
    mode ='w',
    index= False)
(BodyMarkDown_appended.head(7))

(7, 301)


,0_BodyMarkdown,1_BodyMarkdown,2_BodyMarkdown,3_BodyMarkdown,4_BodyMarkdown,5_BodyMarkdown,6_BodyMarkdown,7_BodyMarkdown,8_BodyMarkdown,9_BodyMarkdown,...,291_BodyMarkdown,292_BodyMarkdown,293_BodyMarkdown,294_BodyMarkdown,295_BodyMarkdown,296_BodyMarkdown,297_BodyMarkdown,298_BodyMarkdown,299_BodyMarkdown,id
0,-0.071879,0.034939,-0.053535,-0.007968,-0.009871,0.051640,0.041489,-0.004621,0.119302,-0.023557,...,0.011671,0.007090,-0.123104,0.063124,0.013623,-0.011971,-0.023992,-0.087915,-0.033759,8844866
0,-0.060440,0.093380,-0.036407,-0.027118,0.000425,-0.017460,-0.046280,0.049422,-0.012093,-0.037352,...,0.060340,-0.017930,-0.062969,0.022482,0.038964,0.009898,-0.001850,-0.022312,-0.021957,2659366
0,-0.034124,0.024924,-0.047072,-0.008710,-0.034748,0.027694,-0.057120,0.001856,0.018673,-0.010602,...,-0.028228,0.048713,-0.047364,-0.011032,0.004505,0.007254,-0.006658,-0.042660,0.009221,10940049
0,-0.079857,0.089469,-0.052470,0.026044,0.003003,0.059799,0.007606,-0.020565,0.032908,-0.027515,...,0.011755,0.029449,-0.011121,0.084332,0.031011,-0.034971,0.012470,-0.028939,0.046105,7776743
0,-0.054652,0.078058,-0.055095,-0.041828,-0.006715,0.019636,-0.039402,0.015707,-0.003959,0.019515,...,0.053439,0.027172,-0.026234,0.051671,-0.019979,-0.011625,-0.018657,-0.035646,0.025895,679979
0,-0.094000,0.082542,-0.041527,0.058280,-0.045604,0.062993,-0.040898,0.003576,0.027184,-0.024684,...,0.024547,-0.004932,-0.080273,-0.000455,-0.043256,-0.002785,0.003154,-0.049467,-0.006905,11465224
0,-0.066644,0.068971,-0.001054,-0.024719,0.004286,-0.000992,-0.007656,0.031489,0.025537,-0.025766,...,-0.034858,-0.016276,-0.029664,0.027106,0.027174,0.013064,0.008896,-0.007064,0.022437,47862


In [ ]:
target = get_target(df,"OpenStatus")
target.head()
target.to_hdf(
    "../input/tiny_data/target_glove.h5",
    key='dataset',
    mode ='w',
    index= False)

#### Merge the two embeddings

In [143]:
tt=pd.read_hdf("../input/tiny_data/title.h5",key='dataset',mode ='r')
tb=pd.read_hdf("../input/tiny_data/BodyMarkDown.h5",key='dataset',mode ='r')
tb.head()

,0_BodyMarkdown,1_BodyMarkdown,2_BodyMarkdown,3_BodyMarkdown,4_BodyMarkdown,5_BodyMarkdown,6_BodyMarkdown,7_BodyMarkdown,8_BodyMarkdown,9_BodyMarkdown,...,291_BodyMarkdown,292_BodyMarkdown,293_BodyMarkdown,294_BodyMarkdown,295_BodyMarkdown,296_BodyMarkdown,297_BodyMarkdown,298_BodyMarkdown,299_BodyMarkdown,id
0,-0.071879,0.034939,-0.053535,-0.007968,-0.009871,0.051640,0.041489,-0.004621,0.119302,-0.023557,...,0.011671,0.007090,-0.123104,0.063124,0.013623,-0.011971,-0.023992,-0.087915,-0.033759,8844866
0,-0.060440,0.093380,-0.036407,-0.027118,0.000425,-0.017460,-0.046280,0.049422,-0.012093,-0.037352,...,0.060340,-0.017930,-0.062969,0.022482,0.038964,0.009898,-0.001850,-0.022312,-0.021957,2659366
0,-0.034124,0.024924,-0.047072,-0.008710,-0.034748,0.027694,-0.057120,0.001856,0.018673,-0.010602,...,-0.028228,0.048713,-0.047364,-0.011032,0.004505,0.007254,-0.006658,-0.042660,0.009221,10940049
0,-0.079857,0.089469,-0.052470,0.026044,0.003003,0.059799,0.007606,-0.020565,0.032908,-0.027515,...,0.011755,0.029449,-0.011121,0.084332,0.031011,-0.034971,0.012470,-0.028939,0.046105,7776743
0,-0.054652,0.078058,-0.055095,-0.041828,-0.006715,0.019636,-0.039402,0.015707,-0.003959,0.019515,...,0.053439,0.027172,-0.026234,0.051671,-0.019979,-0.011625,-0.018657,-0.035646,0.025895,679979


In [144]:
tt.head()

,0_Title,1_Title,2_Title,3_Title,4_Title,5_Title,6_Title,7_Title,8_Title,9_Title,...,291_Title,292_Title,293_Title,294_Title,295_Title,296_Title,297_Title,298_Title,299_Title,id
0,-0.065509,0.103647,-0.035116,-0.032617,-0.075866,0.025417,-0.054854,0.020880,0.068233,-0.022143,...,0.032550,0.006832,-0.035522,0.076960,0.012868,-0.033474,0.014736,-0.055689,-0.018701,8844866
0,-0.072350,0.148529,-0.025573,0.024086,-0.033714,0.041064,-0.040374,0.017315,-0.018273,0.017804,...,0.069499,0.008832,-0.038142,0.103691,0.055951,-0.002821,0.000312,0.005471,-0.022646,2659366
0,-0.066269,0.116832,-0.078549,-0.000290,-0.058342,0.018301,-0.042047,-0.020272,-0.002478,0.040167,...,0.023469,0.069197,-0.041510,0.042437,0.078289,-0.033847,0.010500,-0.080802,0.011414,10940049
0,0.004414,0.114957,-0.047366,-0.001067,-0.020975,0.029123,-0.010478,0.007674,-0.014058,0.028262,...,0.038866,0.062495,-0.007892,0.101400,0.088833,-0.005941,-0.007884,0.004912,0.022956,7776743
0,-0.053857,0.123921,-0.050572,-0.027107,-0.054316,0.014609,-0.058074,-0.003075,0.004396,0.042438,...,0.062737,0.041506,-0.042645,0.079617,0.052966,0.005897,-0.002458,-0.034936,0.025330,679979


In [166]:
full_data=tt.merge(tb,how='left',left_on='id',right_on='id')
print(full_data.shape)
full_data=full_data.merge(target,how='left',left_on='id',right_on='id')
print(full_data.shape)

(7, 601)
(7, 602)


In [168]:
full_data.head()

,0_Title,1_Title,2_Title,3_Title,4_Title,5_Title,6_Title,7_Title,8_Title,9_Title,...,291_BodyMarkdown,292_BodyMarkdown,293_BodyMarkdown,294_BodyMarkdown,295_BodyMarkdown,296_BodyMarkdown,297_BodyMarkdown,298_BodyMarkdown,299_BodyMarkdown,target
0,-0.065509,0.103647,-0.035116,-0.032617,-0.075866,0.025417,-0.054854,0.020880,0.068233,-0.022143,...,0.011671,0.007090,-0.123104,0.063124,0.013623,-0.011971,-0.023992,-0.087915,-0.033759,open
1,-0.072350,0.148529,-0.025573,0.024086,-0.033714,0.041064,-0.040374,0.017315,-0.018273,0.017804,...,0.060340,-0.017930,-0.062969,0.022482,0.038964,0.009898,-0.001850,-0.022312,-0.021957,open
2,-0.066269,0.116832,-0.078549,-0.000290,-0.058342,0.018301,-0.042047,-0.020272,-0.002478,0.040167,...,-0.028228,0.048713,-0.047364,-0.011032,0.004505,0.007254,-0.006658,-0.042660,0.009221,open
3,0.004414,0.114957,-0.047366,-0.001067,-0.020975,0.029123,-0.010478,0.007674,-0.014058,0.028262,...,0.011755,0.029449,-0.011121,0.084332,0.031011,-0.034971,0.012470,-0.028939,0.046105,open
4,-0.053857,0.123921,-0.050572,-0.027107,-0.054316,0.014609,-0.058074,-0.003075,0.004396,0.042438,...,0.053439,0.027172,-0.026234,0.051671,-0.019979,-0.011625,-0.018657,-0.035646,0.025895,open


In [167]:
full_data.id.values

array([ 8844866,  2659366, 10940049,  7776743,   679979, 11465224,
          47862])

In [ ]:
if __name__=="__main__":

    t0_total =time.time()

    # Read embeddings
    t0=time.time()
    print("Loading embeddings")
    embeddings = load_vectors("../input/crawl-300d-2M.vec")  
    
    t1 =time.time()
    total_time=t1-t0
    print("time to load", total_time)
    print(" ")
    
    t0=time.time()

    # Read trainig data
    df= pd.read_hdf('../input/tiny_data/train_tiny.h5',key='dataset')
    df = df[["PostId","Title","BodyMarkdown","OpenStatus"]]
    y = df.OpenStatus.values

    t1 =time.time()
    total_time=t1-t0
    print("time to read", total_time)
    print(" ")

    # create a new column called fold and fill it with -1
    df["kfold"] = -1

    t0=time.time()
    print("creating sentence embedding")

    # First column
    vector1 =[]

    print("the BodyMarkDown column embedding")
    
    vector1 = get_vectors(vector1,df.BodyMarkdown.values, embeddings)
    
    vector1 = np.array(vector1)    
    
    print("1st column shape",vector1.shape)

    # save vector1 and target
    save("../input/tiny_data/vector1.npy",vector1)  
    
    save("../input/tiny_data/target.npy",y)

    # second column

    del vector1
    vector1 =[]
    t1 =time.time()
    total_time=t1-t0
    print("time to BodyMarkDown", total_time)
    print(" ")
    print("the Title column embedding")
    t0=time.time()
    vector1 = get_vectors(vector1,df.Title.values,embeddings)
    vector1 = np.array(vector1)    
    print("2nd column shape",vector1.shape)
    save("../input/tiny_data/vector2.npy",vector1)
    t1 =time.time()
    total_time=t1-t0
    print("time to Title", total_time)
    print(" ")

    t1_total =time.time()
    print(f"total time for the process {df.shape},{t1_total-t0_total}")

